In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_feather('cleaned_vehicle_data.feather')
df.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015.0,kia,sorento,lx,suv,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,2014-12-16 04:30:00+00:00
1,2015.0,kia,sorento,lx,suv,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,2014-12-16 04:30:00+00:00
2,2014.0,bmw,3 series,other,sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,2015-01-14 20:30:00+00:00
3,2015.0,volvo,s60,t5,sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,2015-01-28 20:30:00+00:00
4,2014.0,bmw,6 series gran coupe,650i,sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),14570.0,13840.0,2014-12-18 04:30:00+00:00


In [6]:
# Vehicle Age
df['vehicle_age'] = df['saledate'].dt.year - df['year']

In [5]:
# Price Deviation from MMR
df['price_diff'] = df['sellingprice'] - df['mmr']

In [7]:
# 3. Season Buckets
month_to_season = {
    12: 'winter', 1: 'winter', 2: 'winter',
    3: 'spring', 4: 'spring', 5: 'summer',
    6: 'summer', 7: 'monsoon', 8: 'monsoon',
    9: 'fall', 10: 'fall', 11: 'winter'
}
df['season'] = df['saledate'].dt.month.map(month_to_season)

In [9]:
# 4. High Mileage Flag
df['high_mileage'] = df['odometer'] > 100000

In [16]:
# 5. Condition Binning
df['condition_bin'] = pd.cut(df['condition'], bins=[0, 25, 35, 45, 51],
                                   labels=['poor', 'fair', 'good', 'excellent'])

In [11]:
# 6. Popular Color Flag
popular_colors = ['white', 'black', 'gray', 'silver']
df['popular_color'] = df['color'].isin(popular_colors)

In [12]:
# 7. Luxury Make Flag
luxury_makes = ['bmw', 'mercedes-benz', 'audi', 'lexus', 'volvo']
df['is_luxury_make'] = df['make'].str.lower().isin(luxury_makes)

In [13]:
# 8. Interior Type Simplification
df['interior_type'] = df['interior'].replace({'unknown': 'other'})
df['interior_type'] = df['interior_type'].where(df['interior_type'].isin(['black', 'beige', 'gray']), 'other')

In [18]:
# 9. Sale Month Extraction
df['sale_month'] = df['sale_month'].astype('Int64')

In [20]:
df = df.dropna(subset=['saledate'])

In [24]:
cat_cols = ['make', 'model', 'body', 'color', 'transmission', 'interior_type', 'state', 'season', 'condition_bin']
df[cat_cols] = df[cat_cols].astype('category')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 546283 entries, 0 to 550296
Data columns (total 25 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   year            546283 non-null  float64            
 1   make            546283 non-null  category           
 2   model           546283 non-null  category           
 3   trim            546283 non-null  object             
 4   body            546283 non-null  category           
 5   transmission    546283 non-null  category           
 6   vin             546283 non-null  object             
 7   state           546283 non-null  category           
 8   condition       546283 non-null  float64            
 9   odometer        546283 non-null  float64            
 10  color           546283 non-null  category           
 11  interior        546283 non-null  object             
 12  seller          546283 non-null  object             
 13  mmr             546

In [26]:
from scipy.stats import f_oneway

# Group selling prices by season
season_groups = [group['sellingprice'].values for name, group in df.groupby('season')]

f_stat, p_val = f_oneway(*season_groups)

print(f"F-statistic: {f_stat:.2f}")
print(f"P-value: {p_val:.4f}")

if p_val < 0.05:
    print("Statistically significant difference in selling prices across seasons.")
else:
    print("No statistically significant difference in selling prices across seasons.")

/tmp/ipykernel_231/2511342992.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  season_groups = [group['sellingprice'].values for name, group in df.groupby('season')]


F-statistic: 756.06
P-value: 0.0000
Statistically significant difference in selling prices across seasons.


### ANOVA Results: Selling Price by Season

The one-way ANOVA test yielded the following results:

- **F-statistic**: 756.06  
- **P-value**: 0.0000

This indicates a **statistically significant difference** in mean selling prices across seasons. In other words, the time of year when a vehicle is sold has a measurable impact on its price.

This insight supports the inclusion of `season` as a predictive feature in modelling and suggests that pricing strategies should account for seasonal trends.

In [27]:
# Group selling prices by condition_bin
condition_groups = [group['sellingprice'].values for name, group in df.groupby('condition_bin')]

f_stat, p_val = f_oneway(*condition_groups)

print(f"F-statistic: {f_stat:.2f}")
print(f"P-value: {p_val:.4f}")

if p_val < 0.05:
    print("Statistically significant difference in selling prices across condition bins.")
else:
    print("No statistically significant difference in selling prices across condition bins.")

/tmp/ipykernel_231/2484009292.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  condition_groups = [group['sellingprice'].values for name, group in df.groupby('condition_bin')]


F-statistic: 40431.41
P-value: 0.0000
Statistically significant difference in selling prices across condition bins.


### ANOVA Results: Selling Price by Condition

The one-way ANOVA test yielded the following results:

- **F-statistic**: 40,431.41  
- **P-value**: 0.0000

This confirms a **statistically significant difference** in mean selling prices across vehicle condition bins (`poor`, `fair`, `good`, `excellent`). In other words, vehicle condition is a strong driver of price variation.

This insight supports the inclusion of `condition_bin` as a key feature in modeling and pricing strategy. It also opens the door for interaction features — such as combining condition with brand or interior type — to capture compound effects.

In [28]:
from scipy.stats import ttest_ind

# Split selling prices by popular_color flag
popular_prices = df[df['popular_color'] == True]['sellingprice']
nonpopular_prices = df[df['popular_color'] == False]['sellingprice']

t_stat, p_val = ttest_ind(popular_prices, nonpopular_prices, equal_var=False)

print(f"T-statistic: {t_stat:.2f}")
print(f"P-value: {p_val:.4f}")

if p_val < 0.05:
    print("Statistically significant difference in selling prices between popular and non-popular colors.")
else:
    print("No statistically significant difference in selling prices between popular and non-popular colors.")

T-statistic: 57.27
P-value: 0.0000
Statistically significant difference in selling prices between popular and non-popular colors.


### t-Test Results: Selling Price by Popular Color

The two-sample t-test comparing selling prices between popular and non-popular vehicle colours yielded:

- **T-statistic**: 57.27  
- **P-value**: 0.0000

This confirms a **statistically significant difference** in mean selling prices between the two groups. Vehicles with popular colours (`white`, `black`, `gray`, `silver`) tend to sell at different price points compared to those with less common colours.

This insight supports the inclusion of `popular_color` as a binary feature in modelling and suggests that colour preference plays a role in buyer behaviour and market valuation.

In [29]:
# Split selling prices by luxury brand flag
luxury_prices = df[df['is_luxury_make'] == True]['sellingprice']
nonluxury_prices = df[df['is_luxury_make'] == False]['sellingprice']

t_stat, p_val = ttest_ind(luxury_prices, nonluxury_prices, equal_var=False)

print(f"T-statistic: {t_stat:.2f}")
print(f"P-value: {p_val:.4f}")

if p_val < 0.05:
    print("Statistically significant difference in selling prices between luxury and non-luxury brands.")
else:
    print("No statistically significant difference in selling prices between luxury and non-luxury brands.")

T-statistic: 102.99
P-value: 0.0000
Statistically significant difference in selling prices between luxury and non-luxury brands.


### t-Test Results: Selling Price by Luxury Brand

The two-sample t-test comparing selling prices between luxury and non-luxury vehicle brands yielded:

- **T-statistic**: 102.99  
- **P-value**: 0.0000

This confirms a **statistically significant difference** in mean selling prices between the two groups. Luxury brands (`BMW`, `Mercedes-Benz`, `Audi`, `Lexus`, `Volvo`) consistently sell at different price points than non-luxury brands.

This insight supports the inclusion of `is_luxury_make` as a binary feature in modeling and highlights the role of brand prestige in market valuation.

In [31]:
# Create interaction feature
df['luxury_condition'] = df['is_luxury_make'].astype(str) + "_" + df['condition_bin'].astype(str)

# Cast to category for memory efficiency
df['luxury_condition'] = df['luxury_condition'].astype('category')

df['luxury_condition'].value_counts().head()

luxury_condition
False_good         170726
False_poor         140995
False_fair         132437
False_excellent     44024
True_good           23984
Name: count, dtype: int64

In [32]:
# Group selling prices by luxury_condition
lux_cond_groups = [group['sellingprice'].values for name, group in df.groupby('luxury_condition')]

f_stat, p_val = f_oneway(*lux_cond_groups)

print(f"F-statistic: {f_stat:.2f}")
print(f"P-value: {p_val:.4f}")

if p_val < 0.05:
    print("Statistically significant difference in selling prices across luxury-condition combinations.")
else:
    print("No statistically significant difference in selling prices across luxury-condition combinations.")

/tmp/ipykernel_231/3184106952.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  lux_cond_groups = [group['sellingprice'].values for name, group in df.groupby('luxury_condition')]


F-statistic: 20253.36
P-value: 0.0000
Statistically significant difference in selling prices across luxury-condition combinations.


### ANOVA Results: Selling Price by Luxury × Condition

The one-way ANOVA test comparing selling prices across the `luxury_condition` interaction groups yielded:

- **F-statistic**: 20,253.36  
- **P-value**: 0.0000

This confirms a **statistically significant difference** in mean selling prices across combinations of brand prestige and vehicle condition. In other words, the impact of vehicle condition on price is not uniform — it varies depending on whether the vehicle is a luxury brand.

This insight supports the inclusion of `luxury_condition` as an interaction feature in modelling and highlights the importance of compound effects in pricing strategy.

In [33]:
# Create interaction feature
df['season_state'] = df['season'].astype(str) + "_" + df['state'].astype(str)

# Cast to category
df['season_state'] = df['season_state'].astype('category')

df['season_state'].value_counts().head()

season_state
winter_ca    50000
winter_fl    49135
winter_pa    31834
winter_tx    28432
summer_fl    23427
Name: count, dtype: int64

In [34]:
# Group selling prices by season_state
season_state_groups = [group['sellingprice'].values for name, group in df.groupby('season_state')]

f_stat, p_val = f_oneway(*season_state_groups)

print(f"F-statistic: {f_stat:.2f}")
print(f"P-value: {p_val:.4f}")

if p_val < 0.05:
    print("Statistically significant difference in selling prices across season-state combinations.")
else:
    print("No statistically significant difference in selling prices across season-state combinations.")

/tmp/ipykernel_231/490960263.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  season_state_groups = [group['sellingprice'].values for name, group in df.groupby('season_state')]


F-statistic: 261.27
P-value: 0.0000
Statistically significant difference in selling prices across season-state combinations.


### ANOVA Results: Selling Price by Season × State

The one-way ANOVA test comparing selling prices across `season_state` interaction groups yielded:

- **F-statistic**: 261.27  
- **P-value**: 0.0000

This confirms a **statistically significant difference** in mean selling prices across combinations of season and state. In other words, seasonal pricing trends vary by region — a critical insight for location-aware pricing models and regional strategy.

In [35]:
# Create age-mileage ratio
df['age_mileage_ratio'] = df['odometer'] / (df['vehicle_age'] + 1)

# Handle infinite or NaN values
df['age_mileage_ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)
df['age_mileage_ratio'].fillna(df['age_mileage_ratio'].median(), inplace=True)

df['age_mileage_ratio'].describe()

count    546283.000000
mean      11620.280519
std        5640.416067
min           0.055556
25%        7774.236111
50%       10877.369318
75%       14669.873016
max      100449.000000
Name: age_mileage_ratio, dtype: float64

In [36]:
from scipy.stats import pearsonr

# Run Pearson correlation
corr_coef, p_val = pearsonr(df['age_mileage_ratio'], df['sellingprice'])

print(f"Correlation coefficient: {corr_coef:.4f}")
print(f"P-value: {p_val:.4f}")

if p_val < 0.05:
    print("Statistically significant correlation between age-mileage ratio and selling price.")
else:
    print("No statistically significant correlation between age-mileage ratio and selling price.")

Correlation coefficient: -0.1434
P-value: 0.0000
Statistically significant correlation between age-mileage ratio and selling price.


### Correlation Results: Age-Mileage Ratio vs. Selling Price

The Pearson correlation test between `age_mileage_ratio` and `sellingprice` yielded:

- **Correlation coefficient**: -0.1434
- **P-value**: 0.0000

This indicates a **statistically significant negative correlation** — vehicles with higher mileage relative to age tend to sell for lower prices. This validates `age_mileage_ratio` as a meaningful continuous feature for modelling depreciation and wear-and-tear.

In [39]:
# Select continuous features
cont_features = ['odometer', 'vehicle_age', 'price_diff', 'age_mileage_ratio', 'sellingprice', 'mmr', 'sale_month' ]

# Compute correlation matrix
corr_matrix = df[cont_features].corr()

# Display sorted correlations with sellingprice
corr_with_price = corr_matrix['sellingprice'].drop('sellingprice').sort_values(ascending=False)
corr_with_price

mmr                  0.972630
price_diff           0.182097
sale_month          -0.044943
age_mileage_ratio   -0.143361
odometer            -0.624225
vehicle_age         -0.633517
Name: sellingprice, dtype: float64

### Correlation Summary: Continuous Features vs. Selling Price

We computed Pearson correlation coefficients between `sellingprice` and all candidate continuous features. This helps identify which numeric predictors have strong linear relationships with price.

| Feature             | Correlation | Interpretation |
|---------------------|-------------|----------------|
| `mmr`               | +0.97       | Strong positive correlation — vehicles priced above MMR tend to sell higher. |
| `price_diff`        | +0.18       | Mild positive correlation — larger price gaps may reflect premium listings. |
| `sale_month`        | -0.04       | Negligible correlation — month of sale has minimal linear impact. |
| `age_mileage_ratio` | -0.14       | Mild negative correlation — higher wear per year lowers price. |
| `odometer`          | -0.62       | Strong negative correlation — higher mileage reduces price. |
| `vehicle_age`       | -0.63       | Strong negative correlation — older vehicles sell for less. |

### Takeaways:
- `mmr`, `odometer`, and `vehicle_age` are high-impact continuous features.
- `price_diff` and `age_mileage_ratio` offer a moderate signal and may capture nuanced effects.
- `sale_month` shows weak correlation and may be better modelled as categorical or seasonal.

In [41]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder

# Encode categorical features numerically
cat_features = ['season', 'condition_bin', 'popular_color', 'is_luxury_make', 'luxury_condition', 'season_state']
X_cat = df[cat_features].apply(LabelEncoder().fit_transform)
y = df['sellingprice']

# Run ANOVA F-test
selector = SelectKBest(score_func=f_classif, k='all')
selector.fit(X_cat, y)

# Display scores
f_scores = selector.scores_
feature_scores = pd.Series(f_scores, index=cat_features).sort_values(ascending=False)
print(feature_scores)

is_luxury_make      20.653857
condition_bin       13.305771
luxury_condition    12.910421
popular_color        2.445462
season_state         1.563857
season               1.542463
dtype: float64


### ANOVA F-Test Scores: Categorical Features vs. Selling Price

We computed ANOVA F-test scores to evaluate how much each categorical feature explains variance in the target variable `sellingprice`.

| Feature            | F-Score | Interpretation |
|--------------------|---------|----------------|
| `is_luxury_make`   | 20.65   | Strong signal — luxury brand status is a key price driver. |
| `condition_bin`    | 13.31   | High impact — vehicle condition strongly influences price. |
| `luxury_condition` | 12.91   | Compound effect — brand × condition interaction adds value. |
| `popular_color`    | 2.45    | Mild signal — color preference has some pricing influence. |
| `season_state`     | 1.56    | Weak signal — regional seasonality may be noisy. |
| `season`           | 1.54    | Weak signal — season alone has limited explanatory power. |

### Takeaways:
- `is_luxury_make`, `condition_bin`, and `luxury_condition` are high-impact categorical features.
- `popular_color` offers a moderate signal and may be useful in combination.
- `season_state` and `season` show weak standalone impact — consider using them only in interaction terms.

In [45]:
# Define selected features
selected_features = [
    'mmr', 'vehicle_age', 'odometer', 'price_diff', 'age_mileage_ratio',
    'is_luxury_make', 'condition_bin', 'luxury_condition', 'popular_color',
    'sellingprice'
]

df[selected_features].to_feather('model_ready_data.feather')